# 🚀 Train Rasa NLU với PhoBERT-large trên Google Colab

Notebook này sẽ tự động:
1. Cài đặt dependencies
2. Tải PhoBERT-large model
3. Train NLU model
4. Download model về máy local

## ⚠️ Lưu ý Quan Trọng

**Tên thư mục sau khi clone là `ciesta-assistant` (không phải `ciesta-asisstant`)**

## 📋 Yêu cầu
- Clone repository từ GitHub
- Hoặc upload các file cần thiết vào Colab

## 🔧 Cách sử dụng
1. Chạy cell đầu tiên để clone repo
2. Chạy các cell tiếp theo để train
3. Chờ training hoàn tất (30 phút - 2 giờ)
4. Model sẽ tự động được tải về máy local


## Bước 1: Upload files (Nếu chưa có trong Colab)

Nếu bạn đã clone repo thì bỏ qua bước này.


In [ ]:
# Upload files nếu cần
# from google.colab import files
# uploaded = files.upload()


## Bước 2: Clone repository và Setup

**QUAN TRỌNG:** Tên thư mục là `ciesta-assistant` (không phải `ciesta-asisstant`)


In [ ]:
# ============================================
# SETUP TỰ ĐỘNG - CHẠY CELL NÀY ĐẦU TIÊN
# ============================================

import os

# Bước 1: Clone repository
if not os.path.exists("ciesta-assistant"):
    print("📦 Đang clone repository...")
    !git clone https://github.com/HoangPhucDE/ciesta-assistant.git
    print("✅ Đã clone repository thành công")
else:
    print("✅ Repository đã tồn tại")

# Bước 2: Chuyển vào thư mục
%cd ciesta-assistant
print(f"✅ Đã chuyển vào: {os.getcwd()}")

# Bước 3: Kiểm tra files cần thiết
print("\n📋 Kiểm tra files cần thiết:")
required_files = [
    "config.yml",
    "data/nlu.yml",
    "custom_components/phobert_featurizer.py",
    "requirements.txt"
]

missing = []
for file in required_files:
    if os.path.exists(file):
        print(f"  ✅ {file}")
    else:
        print(f"  ❌ {file} - KHÔNG TÌM THẤY")
        missing.append(file)

if missing:
    print(f"\n⚠ Cảnh báo: Thiếu các file: {', '.join(missing)}")
else:
    print("\n✅ Tất cả files cần thiết đã có sẵn!")
    print("\n🚀 Sẵn sàng để train! Chạy cell tiếp theo để bắt đầu.")


## Bước 3: Chạy script training tự động

Script sẽ tự động:
- Cài đặt dependencies
- Tải PhoBERT-large
- Train NLU model
- Download model về máy


In [ ]:
# Chạy script training
!python scripts/training/train_on_colab.py


## Bước 4: Training thủ công (Nếu cần)

Nếu script tự động không hoạt động, bạn có thể chạy từng bước thủ công:


In [ ]:
# Cài đặt dependencies
%pip install -q -r requirements.txt


In [ ]:
# Tải PhoBERT-large model
from huggingface_hub import snapshot_download
import os

os.makedirs("models_hub/phobert-large", exist_ok=True)

snapshot_download(
    repo_id="vinai/phobert-large",
    local_dir="models_hub/phobert-large",
    local_dir_use_symlinks=False,
    resume_download=True
)

print("✅ Đã tải model thành công")


In [ ]:
# Tạo symlink hoặc copy model
import os
import shutil

os.makedirs("models", exist_ok=True)

if os.path.exists("models/phobert-large"):
    if os.path.islink("models/phobert-large"):
        os.unlink("models/phobert-large")
    else:
        shutil.rmtree("models/phobert-large")

try:
    os.symlink("../models_hub/phobert-large", "models/phobert-large")
    print("✅ Đã tạo symlink")
except:
    shutil.copytree("models_hub/phobert-large", "models/phobert-large")
    print("✅ Đã copy model")


In [ ]:
# Train NLU model
!rasa train nlu


In [ ]:
# Download model về máy local
from google.colab import files
from pathlib import Path

models_dir = Path("models")
model_files = list(models_dir.glob("*.tar.gz"))

if model_files:
    # Get latest model
    latest_model = max(model_files, key=lambda x: x.stat().st_mtime)
    print(f"📦 Model mới nhất: {latest_model.name}")
    print(f"📊 Kích thước: {latest_model.stat().st_size / (1024*1024):.2f} MB")
    
    files.download(str(latest_model))
    print("✅ Đã bắt đầu tải model về máy")
else:
    print("❌ Không tìm thấy model")


## 🔍 Troubleshooting

### Lỗi: Out of Memory
- Giảm `batch_size` trong `config.yml`
  ```yaml
  batch_size: [8, 16]  # Thay vì [16, 32]
  ```

### Lỗi: Training quá lâu
- Giảm `epochs` trong `config.yml`
  ```yaml
  epochs: 300  # Thay vì 600
  ```

### Lỗi: Không tìm thấy model
- Kiểm tra đường dẫn trong `config.yml`:
  ```yaml
  model_name: "models/phobert-large"
  ```

### Lỗi: Import error
- Chạy lại cell cài đặt dependencies
- Kiểm tra `requirements.txt` có đầy đủ không
